In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin

# Cargar datos
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
test_ids = test["PassengerId"]

# Combinar para Feature Engineering
full_data = pd.concat([train, test], sort=False)

# Feature Engineering
full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
full_data["Title"] = full_data["Title"].replace(['Mlle', 'Ms'], 'Miss')
full_data["Title"] = full_data["Title"].replace(['Mme'], 'Mrs')
full_data["Title"] = full_data["Title"].replace(['Countess','Lady','Sir','Jonkheer','Don','Dona','Capt','Col','Major','Dr','Rev'], 'Rare')

full_data["FamilySize"] = full_data["SibSp"] + full_data["Parch"] + 1
full_data["IsAlone"] = (full_data["FamilySize"] == 1).astype(int)
full_data["CabinInitial"] = full_data["Cabin"].str[0].fillna('U')

# Separar nuevamente
train = full_data[~full_data["Survived"].isna()]
test = full_data[full_data["Survived"].isna()].drop(columns=["Survived"])

# Variables
X = train.drop(columns=["Survived", "Name", "Ticket", "PassengerId", "Cabin"])
y = train["Survived"]
X_test_final = test.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"])

# Columnas categóricas y numéricas
cat_cols = ["Sex", "Embarked", "Title", "CabinInitial", "Pclass"]
num_cols = ["Age", "Fare", "FamilySize", "IsAlone"]

# Preprocesamiento
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# Modelo y pipeline
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Hiperparámetros
param_dist = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__max_depth": [5, 10, 15, None],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4],
    "classifier__max_features": ["sqrt", "log2"]
}

# Validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomizedSearchCV
search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=30,
                            cv=5, verbose=1, scoring="accuracy", n_jobs=-1, random_state=42)
search.fit(X_train, y_train)

# Evaluación
val_preds = search.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

# Predicción final para Kaggle
final_model = search.best_estimator_
final_preds = final_model.predict(X_test_final)

# Guardar archivo
submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": final_preds.astype(int)
})
submission.to_csv("../data/submission.csv", index=False)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


<>:21: SyntaxWarning: invalid escape sequence '\.'
<>:21: SyntaxWarning: invalid escape sequence '\.'
/var/folders/15/59msbdnd3179pytbr6876h9m0000gn/T/ipykernel_6501/3228753346.py:21: SyntaxWarning: invalid escape sequence '\.'
  full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)


Validation Accuracy: 0.8324022346368715


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

# Cargar datos
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
test_ids = test["PassengerId"]

# Unir para ingeniería de variables
full_data = pd.concat([train, test], sort=False)

# Feature engineering
full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
full_data["Title"] = full_data["Title"].replace(['Mlle', 'Ms'], 'Miss')
full_data["Title"] = full_data["Title"].replace(['Mme'], 'Mrs')
full_data["Title"] = full_data["Title"].replace(['Countess','Lady','Sir','Jonkheer','Don','Dona','Capt','Col','Major','Dr','Rev'], 'Rare')

full_data["FamilySize"] = full_data["SibSp"] + full_data["Parch"] + 1
full_data["IsAlone"] = (full_data["FamilySize"] == 1).astype(int)
full_data["CabinInitial"] = full_data["Cabin"].str[0].fillna('U')

full_data["FarePerPerson"] = full_data["Fare"] / full_data["FamilySize"]
full_data["HasCabin"] = full_data["Cabin"].notnull().astype(int)
full_data["Age*Class"] = full_data["Age"] * full_data["Pclass"]
full_data["TicketPrefix"] = full_data["Ticket"].str.extract(r'([A-Za-z]+)', expand=False).fillna("NONE")

# Separar nuevamente train/test
train = full_data[~full_data["Survived"].isna()]
test = full_data[full_data["Survived"].isna()].drop(columns=["Survived"])

# Variables
X = train.drop(columns=["Survived", "Name", "Ticket", "PassengerId", "Cabin"])
y = train["Survived"]
X_test_final = test.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"])

# Columnas categóricas y numéricas
cat_cols = ["Sex", "Embarked", "Title", "CabinInitial", "Pclass"]
num_cols = ["Age", "Fare", "FamilySize", "IsAlone"]

# Preprocesamiento
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# Pipeline con LightGBM
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LGBMClassifier(random_state=42))
])

# Hiperparámetros
# param_dist = {
#     "classifier__n_estimators": [100, 200, 300, 400],
#     "classifier__learning_rate": [0.01, 0.05, 0.1],
#     "classifier__max_depth": [-1, 3, 5, 10],
#     "classifier__num_leaves": [15, 31, 50, 70],
#     "classifier__min_child_samples": [10, 20, 30]
# }

param_dist = {
    "classifier__n_estimators": [200, 300],
    "classifier__learning_rate": [0.01, 0.05],
    "classifier__max_depth": [3, 5],
    "classifier__num_leaves": [15, 25],
    "classifier__min_child_samples": [20, 30]
}

# Split de validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(search.best_estimator_, X, y, cv=cv, scoring='accuracy')
print("Cross-validation Accuracy: %.4f ± %.4f" % (scores.mean(), scores.std()))

# Búsqueda aleatoria
search = RandomizedSearchCV(
    clf,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    verbose=1,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
search.fit(X_train, y_train)

# Validación
val_preds = search.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

# Predicción para Kaggle
final_model = search.best_estimator_
final_preds = final_model.predict(X_test_final)

# Guardar CSV para enviar
submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": final_preds.astype(int)
})
submission.to_csv("../data/submission_lgbm.csv", index=False)

<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:20: SyntaxWarning: invalid escape sequence '\.'
/var/folders/15/59msbdnd3179pytbr6876h9m0000gn/T/ipykernel_6501/3611824952.py:20: SyntaxWarning: invalid escape sequence '\.'
  full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000516 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 219
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastian

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastian